In [ ]:
import pandas as pd
import requests
import numpy as np
import json
from io import StringIO
import time

BASE_DIR='/home/thanuja/Dropbox/Lansing/'

In [ ]:
def call_census_api(lines):
    data = {
        'returntype':'geographies',
        'benchmark': 'Public_AR_Current',
        'vintage': 'Current_Current'
    }


    files = {
        'addressFile': ('addresses.csv', lines)
    }
    #url = 'https://geocoding.geo.census.gov/geocoder/locations/addressbatch' (only lat and lon but no census tract)
    url = 'https://geocoding.geo.census.gov/geocoder/geographies/addressbatch'
    response = requests.post(url, data = data, files = files)
    return response.text

In [ ]:
fec_file_path = BASE_DIR + "fec_cleaned.csv"
fec_df = pd.read_csv(fec_file_path)
fec_df['Mailing Street'] = fec_df['Mailing Street'].str.replace('\n', ' ').replace('_x000D_', ' ')

In [ ]:
BATCH_SIZE = 50
line_count = 0
batch = ''
output_file = open(BASE_DIR + "fec_geocodes.csv", 'wt')  # append mode
raw_input_file = open(BASE_DIR + 'fec_geocodes_input.csv', 'wt')

selected_cols = fec_df[['FEC ID', 'Mailing Street', 'Mailing City', 'Mailing State/Province', 'Mailing Zip/Postal Code']]
for index, row in selected_cols.dropna().iterrows():
    if pd.isna(row['FEC ID']) or row['FEC ID'] == 'nan':
        continue
    line_count += 1
    batch += f"{row['FEC ID']},{row['Mailing Street']},{row['Mailing City']}"
    batch += f",{row['Mailing State/Province']}," #,{row['Mailing Zip/Postal Code']}"
    if line_count >= BATCH_SIZE:
        #print('[' + batch  + ']')
        print(line_count)
        print(batch, file=raw_input_file)
        line_count = 0
        success = False
        while not success:
            try:
                response = call_census_api(batch)
                #print('*', response, '*')
                success = True
                print('SUCCESS')
            except err:
                print(err)
                time.sleep(7)
            output_file.write(response)
        batch = ''
    else:
        batch += '\n'
raw_input_file.close()
output_file.close()
#print(fec_str.to_string(header=False))

In [ ]:
fec_geocodes_path = BASE_DIR + "fec_geocodes.bak.csv"
fec_geo_df = pd.read_csv(fec_geocodes_path)
fec_geo_df = fec_geo_df[fec_geo_df['Match Indicator'] == 'Match']
# 3519
fec_geo_df.groupby()